In [ ]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib
from matplotlib import pyplot as plt
import tqdm
from skimage.transform import resize
import torchvision.transforms as transforms
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)

In [ ]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.relu = nn.ReLU()

        self.features = nn.Sequential(
            nn.Conv2d(1,12,3,1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(12,16,3,1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,32,3,1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(32 * 5 * 8, 120),
            nn.ReLU(),
            nn.Linear(120, 2)
        )
        
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        
        return x

In [ ]:
cnn = CNN()
print(cnn)
cnn.to(device)
print(summary(cnn, (1, 60, 80)))